In [5]:
import json
import fitz
import pandas as pd
import os
from langchain.docstore.document import Document
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map  # or thread_map
from multiprocessing import cpu_count

from multiprocessing import Manager
from functools import partial
import gc

In [2]:
# !kaggle datasets download -d Cornell-University/arxiv

100%|██████████████████████████████████████| 1.17G/1.17G [01:53<00:00, 11.1MB/s]
100%|██████████████████████████████████████| 1.17G/1.17G [01:53<00:00, 11.2MB/s]


In [6]:
# Get a list of dicts and convert into a pandas df.
arxiv_data = []
for line in open('../arxiv-metadata-oai-snapshot.json', 'r'):
    arxiv_data.append(json.loads(line))
df = pd.DataFrame.from_records(arxiv_data)

In [7]:
df_filtered = df[df['categories'].str.contains('cs', na=False)]

In [21]:
df[df["id"]=="1605.09081"]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
737196,1605.09081,Jayanth Koushik,Jayanth Koushik,Understanding Convolutional Neural Networks,Statistical Machine Learning Course Project at...,None,None,None,stat.OT,http://arxiv.org/licenses/nonexclusive-distrib...,Convoulutional Neural Networks (CNNs) exhibi...,"[{'version': 'v1', 'created': 'Mon, 30 May 201...",2016-10-30,"[[Koushik, Jayanth, ]]"


In [ ]:
df[df["id"]==]

In [6]:
len(df_filtered)/len(df)

0.3264255406434721

In [4]:
len(df_filtered)

749538

In [7]:
del arxiv_data
gc.collect()
del df
gc.collect()

22

In [3]:
basedir="/home/soma4/YEONDOO-arxiv-with-version/YEONDOO-arxiv/data/"
year="14"
month="01"
path_data=os.path.join(basedir,year,month)
id_list=os.listdir(path_data)

In [4]:
id_list=[id.split('v')[0] for id in os.listdir(path_data)]

In [18]:
id_list[0]

'1401.4606'

In [153]:
df[df['id']==id_list[2]]['categories'].item().split(' ')

['cs.AI', 'cs.LO']

In [154]:
df[df['id']==id_list[2]]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
493844,1401.4613,Peter Jeavons,"Peter Jeavons, Justyna Petke",Local Consistency and SAT-Solvers,None,"Journal Of Artificial Intelligence Research, V...",10.1613/jair.3531,None,cs.AI cs.LO,http://arxiv.org/licenses/nonexclusive-distrib...,Local consistency techniques such as k-consi...,"[{'version': 'v1', 'created': 'Sat, 18 Jan 201...",2014-01-21,"[[Jeavons, Peter, ], [Petke, Justyna, ]]"


In [146]:
id="1401.0001"

In [138]:
df[df['id']==id]['update_date']

493837    2014-01-21
Name: update_date, dtype: object

In [132]:
papper_id=id
published=df[df['id']==id]['update_date']
title=df[df['id']==id].title.item()

authors_replaced=df[df['id']==id].authors.item().replace('and',',').split(',')
authors=authors_replaced.split(',')
summary=df[df['id']==id].abstract.item()
url="http://arxiv.org/abs/"+id
ref=df[df['id']==id]['journal-ref'].item()

In [148]:
df[df['id']==id].authors.item().replace('and',',').split(',')

['Nils Bertschinger ',
 ' David H. Wolpert ',
 ' Eckehard Olbrich ',
 '\n  Juergen Jost']

In [147]:
df[df['id']==id].authors.item()

'Nils Bertschinger and David H. Wolpert and Eckehard Olbrich and\n  Juergen Jost'

In [141]:
published=df[df['id']==id]['update_date'].item()
published


'2014-01-21'

In [17]:
pdf_list[0][:-6]

'1401.4606'

In [11]:

def Wrapper(shared_list,doc_file_name):
    
    id=doc_file_name.split('v')[0]
    file_path=os.path.join(basedir,year,month,doc_file_name)
    try:
        with fitz.open(file_path) as doc_file:
            text: str = "".join(page.get_text() for page in doc_file)
    except:
        print(file_path)
        return
    result=df_filtered[df_filtered["id"]==id]

    metadata = {
        "Published": str(result["update_date"].item()),
        "Title": result.title.item(),
        "Authors": result.authors.item().replace('and',',').split(','),
        "Summary": result.abstract.item(),

        "paper_id": id,

        "journal_ref": result['journal-ref'].item(),
        
        "categories": result.categories.item().split(' '),
        "source": "http://arxiv.org/abs/"+id,
    }
    doc = Document(
        page_content=text, metadata=metadata
    )
    shared_list.append(doc)

In [ ]:

def Wrapper2(shared_list,inputs):
    
    doc_file_name = inputs[0]
    result=inputs[1]
    id=doc_file_name.split('v')[0]
    file_path=os.path.join(basedir,year,month,doc_file_name)
    with fitz.open(file_path) as doc_file:
        text: str = "".join(page.get_text() for page in doc_file)
    
    # result=df[df["id"]==id]

    metadata = {
        "Published": str(result["update_date"].item()),
        "Title": result.title.item(),
        "Authors": result.authors.item().replace('and',',').split(','),
        "Summary": result.abstract.item(),

        "paper_id": id,

        "journal_ref": result['journal-ref'].item(),
        
        "categories": result.categories,
        "source": "http://arxiv.org/abs/"+id,
    }
    doc = Document(
        page_content=text, metadata=metadata
    )
    shared_list.append(doc)

In [9]:
basedir="/home/soma4/YEONDOO-arxiv-with-version/YEONDOO-arxiv/data/"
year="14"
month="01"
path_data=os.path.join(basedir,year,month)
pdf_list=os.listdir(path_data)

In [ ]:

chunksize=2

manager = Manager()
shared_list = manager.list()
process_map(partial(Wrapper, shared_list),pdf_list,max_workers=5,chunksize=chunksize)

In [13]:
shared_list[0]

Document(page_content='Journal of Artiﬁcial Intelligence Research 43 (2012) 329-351\nSubmitted 11/11; published 03/12\nLocal Consistency and SAT-Solvers\nPeter Jeavons\nPeter.Jeavons@cs.ox.ac.uk\nJustyna Petke\nJustyna.Petke@cs.ox.ac.uk\nDepartment of Computer Science, University of Oxford\nWolfson Building, Parks Road, Oxford, OX1 3QD, UK\nAbstract\nLocal consistency techniques such as k-consistency are a key component of specialised\nsolvers for constraint satisfaction problems.\nIn this paper we show that the power of\nusing k-consistency techniques on a constraint satisfaction problem is precisely captured by\nusing a particular inference rule, which we call negative-hyper-resolution, on the standard\ndirect encoding of the problem into Boolean clauses. We also show that current clause-\nlearning SAT-solvers will discover in expected polynomial time any inconsistency that can\nbe deduced from a given set of clauses using negative-hyper-resolvents of a ﬁxed size. We\ncombine these t

In [40]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(model_name="text-embedding-ada-002",allowed_special={"<|endoftext|>"},chunk_size=250, chunk_overlap=0)

In [41]:
texts = text_splitter.split_documents(shared_list)

In [46]:
len(texts)

23974

In [45]:
len(texts) // 4000

5

In [49]:
r=len(texts) // 4000

In [47]:
len_texts=len(texts)

In [56]:
a=[]
for i in range(r+1):
    start=i*4000
    end=start+4000
    if end>len_texts:
        end=len_texts
    a.append(texts[start:end])

In [42]:
len(texts[0].page_content)

1066

In [57]:
len(a[0])

4000

In [58]:
for aa in a:
    cnt=0
    for aaa in aa:
        cnt+=len(encoding.encode(aaa.page_content))
    print(cnt)

995312
995811
995924
996052
996226
989130


In [43]:
import tiktoken
encoding = tiktoken.encoding_for_model("text-embedding-ada-002")
len(encoding.encode(texts[0].page_content))

250

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts2 = text_splitter2.split_documents(shared_list[:10])

In [27]:
len(texts2[0].page_content)

943

In [40]:
texts[0]

Document(page_content='Journal of Artiﬁcial Intelligence Research 43 (2012) 329-351\nSubmitted 11/11; published 03/12\nLocal Consistency and SAT-Solvers\nPeter Jeavons\nPeter.Jeavons@cs.ox.ac.uk\nJustyna Petke\nJustyna.Petke@cs.ox.ac.uk\nDepartment of Computer Science, University of Oxford\nWolfson Building, Parks Road, Oxford, OX1 3QD, UK\nAbstract\nLocal consistency techniques such as k-consistency are a key component of specialised\nsolvers for constraint satisfaction problems.\nIn this paper we show that the power of\nusing k-consistency techniques on a constraint satisfaction problem is precisely captured by\nusing a particular inference rule, which we call negative-hyper-resolution, on the standard\ndirect encoding of the problem into Boolean clauses. We also show that current clause-\nlearning SAT-solvers will discover in expected polynomial time any inconsistency that can\nbe deduced from a given set of clauses using negative-hyper-resolvents of a ﬁxed size. We', metadata={'Pub

In [32]:
len(shared_list)

201

In [37]:
shared_list[0].page_content

'Journal of Artiﬁcial Intelligence Research 43 (2012) 329-351\nSubmitted 11/11; published 03/12\nLocal Consistency and SAT-Solvers\nPeter Jeavons\nPeter.Jeavons@cs.ox.ac.uk\nJustyna Petke\nJustyna.Petke@cs.ox.ac.uk\nDepartment of Computer Science, University of Oxford\nWolfson Building, Parks Road, Oxford, OX1 3QD, UK\nAbstract\nLocal consistency techniques such as k-consistency are a key component of specialised\nsolvers for constraint satisfaction problems.\nIn this paper we show that the power of\nusing k-consistency techniques on a constraint satisfaction problem is precisely captured by\nusing a particular inference rule, which we call negative-hyper-resolution, on the standard\ndirect encoding of the problem into Boolean clauses. We also show that current clause-\nlearning SAT-solvers will discover in expected polynomial time any inconsistency that can\nbe deduced from a given set of clauses using negative-hyper-resolvents of a ﬁxed size. We\ncombine these two results to show tha

In [24]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def num_tokens_from_doc(shared_list2,doc) -> int:
    """Returns the number of tokens in a text string."""
    string=doc.page_content
    num_tokens = len(encoding.encode(string, allowed_special={"<|endoftext|>"}))
    shared_list2.append(num_tokens)

In [35]:
num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

In [18]:
len(texts)

42080

In [28]:
shared_list2 = manager.list()
process_map(partial(num_tokens_from_doc, shared_list2),texts,max_workers=5,chunksize=chunksize)

  0%|          | 0/42080 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [48]:
cnt=0
for a in shared_list2:
    cnt+=a

In [50]:
(cnt/1000)*0.0001

0.5953371000000001

In [6]:
cnt=10
price=0.001
with open("./b.txt", 'a') as file:
    data = "%d tokens = %lf\n" %(cnt,price)
    file.write(data)

In [ ]:
file_name=os.path.join('./',"TokenPrice")+'.txt'
            with open(file_name, 'a') as file:
                data = "%lf\n" %(price)
                file.write(data)

In [57]:
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import OpenAIEmbeddings
import getpass
os.environ['OPENAI_API_KEY']=getpass.getpass()
embeddings = OpenAIEmbeddings()



docsearch = OpenSearchVectorSearch.from_documents(
    texts[:5],
    embeddings,
    opensearch_url="https://search-yeondoo-opensearch-2r3sj6ok7iowthxkgjrnhxsyv4.ap-northeast-2.es.amazonaws.com",
    http_auth=("admin", "qiQduz-pyrhab-hexzo4"),
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    index_name="allcontent4",# with metadata source
    bulk_size=2000
)

In [71]:
texts[0].metadata['paper_id']

'1401.4613'

In [3]:
f=open("./TokenPrice.txt",'r')
f1 = f.readlines()
price=0
for x in f1:
    price+=float(x)

In [4]:
price

99.64080999999999